In [10]:
import psycopg2
import datetime

In [83]:
timestr = "03:00:00"
times = "0300"

In [84]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [85]:
sql = "CREATE TABLE plist"+times+" (id SERIAL NOT NULL, "
sql += "mesh BIGINT NULL DEFAULT NULL,"
sql += "i INTEGER NULL DEFAULT NULL,"
sql += "j INTEGER NULL DEFAULT NULL,"
sql += "param REAL NULL DEFAULT NULL "
sql += ");"

cur =connection.cursor()
cur.execute(sql)
cur.close()
connection.commit()

In [86]:
#geojson情報を作成する。
sql = "SELECT tb_uni.start_mesh_area,  tb_uni.stI, tb_uni.stJ, SUM( tb_uni.param ) AS summ FROM "
sql += "( "
sql += "SELECT distinct start_mesh_area,  st.i AS stI, st.j AS stJ, cast(kddi_location_0127.population AS FLOAT )  * -1 AS param "
sql += "FROM kddi_location_0127 "
sql += "INNER JOIN pointdata_index AS st ON st.mesh = kddi_location_0127.start_mesh_area "
sql += "WHERE yyyymmdd = 20220401 AND end_time = '"+timestr+"' and odroutetype = 'S' "
sql += "UNION "
sql += "SELECT distinct end_mesh_area,  ed.i AS edI, ed.j AS edJ, cast(kddi_location_0127.population AS FLOAT ) AS param "
sql += "FROM kddi_location_0127  "
sql += "INNER JOIN pointdata_index AS ed ON ed.mesh = kddi_location_0127.end_mesh_area "
sql += "WHERE yyyymmdd = 20220401 AND end_time = '"+timestr+"' and odroutetype = 'S'  "
sql += ") as tb_uni "
sql += "GROUP BY tb_uni.start_mesh_area,  tb_uni.stI, tb_uni.stJ "
sql += "ORDER BY tb_uni.start_mesh_area "
cur =connection.cursor()
#print(sql)
cur.execute(sql)
result = cur.fetchall()
for row in result:
    sql = "INSERT INTO plist"+times+"(id,mesh,i,j,param) VALUES(DEFAULT,"+str(row[0])+","+str(row[1])+","+str(row[2])+","+str(row[3])+");" 
    cur.execute(sql)
connection.commit()
cur.close()

In [87]:
f = open('test'+times+'.json', 'w')
find = 0
f.write('{\n')
f.write('    "nodes": [\n')

sql = "SELECT  'n' || id AS id ,i ,j,param,mesh FROM plist"+times+" "
cur =connection.cursor()
cur.execute(sql)
finda=0
findb=0
for row in cur:
    if find != 0:
        f.write(',')
    f.write('        {\n')
    f.write('            "id": "' + str(row[0]) + '",\n')
    f.write('            "label": "' + str(row[3]) + '",\n')
    f.write('            "x": ' + str(row[2]) + ',\n')
    f.write('            "y": ' + str(row[1]) + ',\n')
    if str(row[4]) == '50326611134':
        finda = 1
        f.write('            "size": 2\n')
    elif str(row[4]) == '50326610134':
        findb = 1
        f.write('            "size": 2\n')
    else:
        f.write('            "size": 1\n')
    f.write('        }\n')
    find = find + 1

if finda == 0:
    if find != 0:
        f.write(',')
    f.write('        {\n')
    f.write('            "id": "50326611134",\n')
    f.write('            "label": "99999",\n')
    f.write('            "x": 175,\n')
    f.write('            "y": 219,\n')
    f.write('            "size": 2\n')
    f.write('        }\n')

if findb == 0:
    if find != 0:
        f.write(',')
    f.write('        {\n')
    f.write('            "id": "50326610134",\n')
    f.write('            "label": "88888",\n')
    f.write('            "x": 167,\n')
    f.write('            "y": 219,\n')
    f.write('            "size": 2\n')
    f.write('        }\n')
f.write('    ],\n')
cur.close()


In [88]:
#geojson情報を作成する。
sql = "SELECT kddi_location_0127.population, pst15.id AS source, ped15.id AS target, kddi_location_0127.start_time "
sql += "FROM kddi_location_0127 "
sql += "INNER JOIN pointdata_index AS st ON st.mesh = kddi_location_0127.start_mesh_area "
sql += "INNER JOIN plist"+times+" AS pst15 ON st.mesh = pst15.mesh "
sql += "INNER JOIN pointdata_index AS ed ON ed.mesh = kddi_location_0127.end_mesh_area "
sql += "INNER JOIN plist"+times+" AS ped15 ON ed.mesh = ped15.mesh "
sql += "WHERE yyyymmdd = 20220401 AND end_time = '"+timestr+"' and odroutetype = 'S' "
cur =connection.cursor()
cur.execute(sql)
find = 0
f.write('    "edges": [\n')
for row in cur:
    if find != 0:
        f.write(',')
    f.write('        {\n')
    f.write('            "id": "e' + str(find) + '",\n')
    f.write('            "type": "arrow",\n')
    f.write('            "source": "n' + str(row[1]) + '",\n')
    f.write('            "target": "n' + str(row[2]) + '",\n')
    f.write('            "label": "' + str(row[0]) +'*' + str(row[3]) + '"\n')
    f.write('        }\n')
    find = find + 1
f.write('    ]\n')
f.write('}\n')
cur.close()
f.close()